# Experimental new pipeline

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
import rpy2.robjects as robjects
import multiprocessing
import re
import os

import numpy as np
from shutil import copyfile

from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline, Tree
from pimp.settings import getString

## Load analysis and project

In [41]:
analysis = Analysis.objects.get_or_create(id=9)[0]

DEBUG 2016-09-14 11:24:04,551 utils.py:91 | (0.000) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 9 LIMIT 21; args=(9,)


In [42]:
project = Project.objects.get_or_create(id=9)[0]

DEBUG 2016-09-14 11:24:04,735 utils.py:91 | (0.000) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 9 LIMIT 21; args=(9,)


In [43]:
pipeline = Rpy2Pipeline(analysis, project, False)

******************************************
Setup rpy2 connection
******************************************


In [6]:
pipeline.setup()

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-14 11:02:22,439 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-14 11:02:22,479 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarn

Setup called
2016-09-14 11:02:22 INFO:Pimp.getPimpWd:Data dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects
2016-09-14 11:02:22 INFO:Pimp.getPimpWd:Project dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9
2016-09-14 11:02:22 INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 9
2016-09-14 11:02:22 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-14 11:02:22 9:INFO:Pimp.validateInput:Input validated correctly!
2016-09-14 11:02:22 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 9
2016-09-14 11:02:22 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-14 11:02:22 9:INFO:Pimp.getAnalysisParams:Setting params
2016-09-14 11:02:22 9:DEBUG:Pimp.getAnalysisParams:Name: iqr Value: 0.5
2016-09-14 11:02:22 9:DEBUG:Pimp.getAnalysisParams:Name: rsd Value: 0.5
2016-09-14 11:02:22 9:DEBUG:Pimp.getAnalysisParams:Name: noise Value: 0.8
2016-09-14 11:02:22 9:DEBUG:Pimp.getAnalysisParams:Name: p

WARNING 2016-09-14 11:02:22,757 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_9

  res = super(Function, self).__call__(*new_args, **new_kwargs)



2016-09-14 11:02:22 9:INFO:Pimp.createAnalysisDir:Analysis dir created at analysis_9


## Peak detection

In [7]:
def get_value(r_vect, key):
    return r_vect[r_vect.names.index(key)]

In [8]:
def get_formatted_value(r_vect, key, analysis_id=None, subst=None):

    str_vect = get_value(r_vect, key)
    my_str = str_vect[0]
    
    # replace 'analysis' with 'analysis_%d'
    if analysis_id is not None:
        my_str = my_str.replace('analysis', 'analysis_%d' % analysis_id)
    
    # replace the 'polarity' bit
    if subst is not None:
        my_str = my_str % subst

    return my_str

In [9]:
def peak_detection(files, xcms_params, n_slaves):

    # collect the parameters
    args = {
        'files'           : files,
        'method'          : get_value(xcms_params, 'method'),
        'ppm'             : get_value(xcms_params, 'ppm'),
        'peakwidth'       : get_value(xcms_params, 'peakwidth'),
        'snthresh'        : get_value(xcms_params, 'snthresh'),
        'prefilter'       : get_value(xcms_params, 'prefilter'),
        'integrate'       : get_value(xcms_params, 'integrate'),
        'mzdiff'          : get_value(xcms_params, 'mzdiff'),
        'verbose.columns' : get_value(xcms_params, 'verbose.columns'),
        'fitgauss'        : get_value(xcms_params, 'fitgauss'),
        'nSlaves'         : n_slaves
    }

    # call centwave
    xcms_set = robjects.r['xcmsSet']    
    xset = xcms_set(**args)
    return xset

In [10]:
positive_files = get_value(pipeline.metadata.r_files, 'positive')
xcms_params = get_value(pipeline.pimp_params, 'xcms.params')
n_slaves = multiprocessing.cpu_count()-1

print positive_files, xcms_params, n_slaves

 [1] "samples/POS/Beer_4_full1.mzXML" "samples/POS/Beer_4_full2.mzXML"
 [3] "samples/POS/Beer_4_full3.mzXML" "samples/POS/Beer_1_full1.mzXML"
 [5] "samples/POS/Beer_1_full2.mzXML" "samples/POS/Beer_1_full3.mzXML"
 [7] "samples/POS/Beer_2_full1.mzXML" "samples/POS/Beer_2_full2.mzXML"
 [9] "samples/POS/Beer_2_full3.mzXML" "samples/POS/Beer_3_full1.mzXML"
[11] "samples/POS/Beer_3_full2.mzXML" "samples/POS/Beer_3_full3.mzXML"
 $method
[1] "centWave"

$ppm
[1] 3

$peakwidth
[1]   5 100

$snthresh
[1] 2

$prefilter
[1]   2 500

$integrate
[1] 1

$mzdiff
[1] 0.001

$verbose.columns
[1] TRUE

$fitgauss
[1] FALSE

 7


In [11]:
xset_pos = peak_detection(positive_files, xcms_params, n_slaves)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘snow’


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-14 11:02:57,444 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘snow’


  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:BiocGenerics’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, parApply, parCapply,
    parLapply, parRapply, parSapply


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-14 11:02:57,447 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The f

Starting snow cluster with 7 local sockets.
Detecting features in file # 1 : Beer_4_full1.mzXML 
Detecting features in file # 2 : Beer_4_full2.mzXML 
Detecting features in file # 3 : Beer_4_full3.mzXML 
Detecting features in file # 4 : Beer_1_full1.mzXML 
Detecting features in file # 5 : Beer_1_full2.mzXML 
Detecting features in file # 6 : Beer_1_full3.mzXML 
Detecting features in file # 7 : Beer_2_full1.mzXML 
Detecting features in file # 8 : Beer_2_full2.mzXML 
Detecting features in file # 9 : Beer_2_full3.mzXML 
Detecting features in file # 10 : Beer_3_full1.mzXML 
Detecting features in file # 11 : Beer_3_full2.mzXML 
Detecting features in file # 12 : Beer_3_full3.mzXML 


## Peak alignment

In [12]:
mzmatch_params = get_value(pipeline.pimp_params, 'mzmatch.params')
peakml_params = get_value(pipeline.pimp_params, 'peakml.params')
mzmatch_outputs = get_value(pipeline.pimp_params, 'mzmatch.outputs')
method = get_formatted_value(mzmatch_params, 'rt.alignment')
print method

obiwarp


In [13]:
print mzmatch_params

$rsd
[1] 0.5

$iqr
[1] 0.5

$rt.alignment
[1] "obiwarp"

$noise
[1] 0.8

$ppm
[1] 3

$order
[1] 3

$maxrt
[1] 120

$rtwindow
[1] 30

$id.rtwindow
[1] 0.05

$combination
[1] "set"

$mindetections
[1] 3

$minintensity
[1] 5000

$adducts.positive
[1] "M+H,M+ACN+Na,M+Na,M+K,M+ACN+H"

$adducts.negative
[1] "M-H,M-ACN+Na,M-Na,M-K,M-ACN+H"




In [14]:
print mzmatch_outputs

$alignment.folder
[1] "analysis/%s/combined_rt.alignment"

$combined.folder
[1] "analysis/%s/combined"

$combined.rsd.filtered.folder
[1] "analysis/%s/combined_RSD_filtered"

$combined.rsd.rejected.folder
[1] "analysis/%s/combined_RSD_rejected"

$final.combined.peakml.file
[1] "analysis/%s/final_combined.peakml"

$final.combined.noise.filtered.file
[1] "analysis/%s/final_combined_nf.peakml"

$final.combined.simple.filtered.file
[1] "analysis/%s/final.combined_sf.peakml"

$final.combined.gapfilled.file
[1] "analysis/%s/final_combined_gapfilled.peakml"

$final.combined.related.file
[1] "analysis/%s/final_combined_related.peakml"

$final.combined.basepeaks.file
[1] "analysis/%s/final_combined_basepeaks.peakml"

$final.combined.related.id.file
[1] "analysis/%s/final_combined_related_identified.peakml"

$final.combined.related.stds.id.file
[1] "analysis/%s/final_combined_related_stds_identified.peakml"

$final.combined.basepeaks.id.file
[1] "analysis/%s/final_combined_basepeaks_identified.p

In [15]:
method = get_value(mzmatch_params, 'rt.alignment')[0]

In [16]:
xset = xset_pos

In [17]:
def rt_correct(xset, method, peakml_params, mzmatch_outputs, n_slaves, verbose):
    
    if method == 'obiwarp':
        retcor = robjects.r['retcor']
        xset_aln = retcor(xset, method='obiwarp', profStep=0.01)        
    elif method == 'loess':
        retcor = robjects.r['retcor']
        xset_aln = retcor(xset, method='loess', family="symmetric")
    else:
        xset_aln = xset
        
    filepaths = robjects.r['filepaths']
    fp = filepaths(xset_aln)

    split = robjects.r['split']
    xseto = split(xset_aln, fp)
    return fp, xseto

In [18]:
fp, xseto = rt_correct(xset, method, peakml_params, mzmatch_outputs, n_slaves, True)

center sample:  Beer_1_full2 
Processing: Beer_4_full1  Beer_4_full2  Beer_4_full3  Beer_1_full1  Beer_1_full3  Beer_2_full1  Beer_2_full2  Beer_2_full3  Beer_3_full1  Beer_3_full2  Beer_3_full3  


#### get java heapsize

In [19]:
xmx = getString('PIMP_JAVA_PARAMETERS', '')
print xmx

found = re.findall(r'\d+', xmx)
heapsize = int(found[0])
print heapsize

-Xmx8192m
8192


#### initialize mzmatch

In [20]:
# I think this should be called only once??
args = {
    'memorysize' : heapsize,
    'version.1'  : False
}
mzmatch_init = robjects.r['mzmatch.init']
mzmatch_init(**args)

2016-09-14 11:13:28 9:INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-14 11:13:28 9:INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx8192m
2016-09-14 11:13:28 9:INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-14 11:13:28 9:INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/swt-4.5-cocoa-macosx-x86_64.jar


<BoolVector - Python:0x115407320 / R:0x7fff40f73ff8>
[       0]

In [21]:
write_peakml = robjects.r['PeakML.xcms.write.SingleInstance']

In [22]:
polarity_folder = get_formatted_value(mzmatch_outputs, 'polarity.folder', analysis_id=analysis.id, subst='pos')

In [23]:
ionisation = get_value(peakml_params, 'ionisation')
add_scans = get_value(peakml_params, 'addscans')
write_rejected = get_value(peakml_params, 'writeRejected')
apodisation_filter = get_value(peakml_params, 'ApodisationFilter')
ppm = get_value(peakml_params, 'ppm') # not set inside peakml_params ..?
if ppm == 0:
    ppm = 5

In [24]:
peakml_files = []
regex = re.compile(re.escape('mzxml'), re.IGNORECASE)
for single_fp in fp:
    replaced = regex.sub('peakml', single_fp)
    outfile = os.path.join(polarity_folder, os.path.basename(replaced))
    peakml_files.append(outfile)

In [25]:
print peakml_files

['analysis_9/pos/Beer_4_full1.peakml', 'analysis_9/pos/Beer_4_full2.peakml', 'analysis_9/pos/Beer_4_full3.peakml', 'analysis_9/pos/Beer_1_full1.peakml', 'analysis_9/pos/Beer_1_full2.peakml', 'analysis_9/pos/Beer_1_full3.peakml', 'analysis_9/pos/Beer_2_full1.peakml', 'analysis_9/pos/Beer_2_full2.peakml', 'analysis_9/pos/Beer_2_full3.peakml', 'analysis_9/pos/Beer_3_full1.peakml', 'analysis_9/pos/Beer_3_full2.peakml', 'analysis_9/pos/Beer_3_full3.peakml']


In [26]:
directory = os.path.dirname(peakml_files[0])
if not os.path.exists(directory):
    os.makedirs(directory)

In [27]:
for i in range(len(peakml_files)):
    print 'Now creating %s' % peakml_files[i]
    write_peakml(xseto[i], outputfile=peakml_files[i], ionisation=ionisation, 
                 addscans=add_scans, writeRejected=write_rejected, 
                 ApodisationFilter=apodisation_filter, ppm=ppm)

Now creating analysis_9/pos/Beer_4_full1.peakml
2016-09-14 11:13:28 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-14 11:13:29 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2016-09-14 11:13:29 9:INFO:PeakML.xcms.write.SingleInstance:- /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/samples/POS/Beer_1_full1.mzXML
2016-09-14 11:13:51 9:INFO::11539 peaks exported.
Now creating analysis_9/pos/Beer_4_full2.peakml
2016-09-14 11:13:51 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-14 11:13:51 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2016-09-14 11:13:51 9:INFO:PeakML.xcms.write.SingleInstance:- /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/samples/POS/Beer_1_full2.mzXML
2016-09-14 11:14:17 9:INFO::11471 peaks exported.
Now creating analysis_9/pos/Beer_4_full3.peakml
2016-09-14 11:14:18 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-14 11:14:18 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data

## Combine

Combine files by group, RSD filter if required (Not convinced we should be using this filter) and combine group files

In [140]:
combined_dir = os.path.join(directory, 'combined')

In [141]:
if not os.path.exists(combined_dir):
    os.makedirs(combined_dir)

In [142]:
factors = pipeline.get_groups()

In [143]:
print factors

[beer_smell with 2 levels, beer_colour with 2 levels, beer_taste with 3 levels]


In [144]:
dims = []
for f in factors:
    print f.label
    for lev in f.levels:
        print ' -', lev, f.level_files[lev]
    dims.append(len(f.levels))

beer_smell
 - smell_good ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3', 'Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']
 - smell_bad ['Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_colour
 - colour_dark ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3']
 - colour_light ['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3', 'Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_taste
 - taste_delicious ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3']
 - taste_okay ['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']
 - taste_awful ['Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']


In [145]:
all_pos_files = pipeline.get_short_names('positive')

In [146]:
mat = np.empty(dims, dtype=object)
print mat.shape

(2, 2, 3)


In [147]:
def populate_tree(parent, factors):
    depth = parent.depth + 1
    if depth > len(factors)-1:
        return
    factor = factors[depth]
    nodes = []
    for level in factor.levels:
        node = Tree(factor, level, depth, parent)
        populate_tree(node, factors)
        
parent = Tree(None, 'root', -1, None)
parent.level_files = set(pipeline.get_short_names('positive'))
populate_tree(parent, factors)

In [148]:
def print_tree(node, mat, coord):
    if node.level_idx is not None:
        coord.append(node.level_idx)
    output = ''.join(['\t' for i in range(node.depth)])
    output += '%d %s %s : %s at %s\n' % (node.depth, node.factor, node.level_label, 
                                         node.level_files, coord)
    mat[tuple(coord)] = node.level_files # coord must be a tuple
    for child in node.children:
        new_coord = list(coord) # copy
        output += print_tree(child, mat, new_coord)
    return output

output = print_tree(parent, mat, [])
print output

-1 None root : set(['Beer_3_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_3_full2', 'Beer_1_full1', 'Beer_3_full3', 'Beer_4_full3', 'Beer_4_full2', 'Beer_4_full1', 'Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2']) at []
0 beer_smell with 2 levels smell_good : set(['Beer_2_full2', 'Beer_1_full3', 'Beer_1_full2', 'Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3', 'Beer_2_full1', 'Beer_2_full3', 'Beer_1_full1']) at [0]
	1 beer_colour with 2 levels colour_dark : set(['Beer_1_full2', 'Beer_1_full3', 'Beer_1_full1', 'Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2']) at [0, 0]
		2 beer_taste with 3 levels taste_delicious : set(['Beer_1_full2', 'Beer_1_full3', 'Beer_1_full1', 'Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2']) at [0, 0, 0]
		2 beer_taste with 3 levels taste_okay : set([]) at [0, 0, 1]
		2 beer_taste with 3 levels taste_awful : set([]) at [0, 0, 2]
	1 beer_colour with 2 levels colour_light : set(['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']) at [0, 1]
		2 beer_taste with 3 levels tas

In [149]:
print mat

[[[ set(['Beer_1_full2', 'Beer_1_full3', 'Beer_1_full1', 'Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2'])
   set([]) set([])]
  [set([]) set(['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']) set([])]]

 [[set([]) set([]) set([])]
  [set([]) set([]) set(['Beer_4_full3', 'Beer_4_full2', 'Beer_4_full1'])]]]


In [171]:
def index_to_string(factors, index):
    assert len(factors) == len(index)
    output = []
    for d in range(len(factors)):
        f = factors[d]
        idx = index[d]
        output.append(f.levels[idx])
    return ','.join(output)

In [182]:
def write_combinations(factors, mat, out_file):
    rows = []
    with open(out_file, 'w') as f:    
        i = 0
        for index, files in np.ndenumerate(mat):

            combi = 'combi_%s' % i
            row = (combi, index, index_to_string(factors, index), list(files))
            rows.append(row)
            
            s = '%s\t%s\t%s\t%s\n' % row
            f.write(s)
            i += 1
    return rows

In [183]:
out_file = os.path.join(combined_dir, 'combined.tsv')
rows = write_combinations(factors, mat, out_file)

In [190]:
for row in rows:
    
    print row
    combi_label, coord, description, files = row
    
    combi_dir = os.path.join(combined_dir, combi_label)
    if not os.path.exists(combi_dir):
        os.makedirs(combi_dir)
    
    # copy files
    for f_name in files:
        abs_dir = os.path.abspath(directory)
        source = os.path.join(abs_dir, f_name + '.peakml')
        target = combi_dir
        copyfile()
        print f_path, '--->', target
    

('combi_0', (0, 0, 0), 'smell_good,colour_dark,taste_delicious', ['Beer_1_full2', 'Beer_1_full3', 'Beer_1_full1', 'Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2'])
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/analysis_9/pos/Beer_1_full2.peakml ---> analysis_9/pos/combined/combi_0
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/analysis_9/pos/Beer_1_full3.peakml ---> analysis_9/pos/combined/combi_0
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/analysis_9/pos/Beer_1_full1.peakml ---> analysis_9/pos/combined/combi_0
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/analysis_9/pos/Beer_2_full1.peakml ---> analysis_9/pos/combined/combi_0
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/analysis_9/pos/Beer_2_full3.peakml ---> analysis_9/pos/combined/combi_0
/Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/analysis_9/pos/Beer_2_full2.peakml ---> analysis_9/pos/combined/combi_0
('combi_1', (0, 0, 1), 'smell_goo